In [1]:
import os
import pandas as pd
import Bio.PDB.PDBList as biopy
import PCN
import traceback
import tarfile
import random
import numpy as np
import networkx as nx
import MDAnalysis as mda
import random
import Bio
import matplotlib.pyplot as plt
import esm
import torch
import os
from Bio import SeqIO
import itertools
from typing import List, Tuple
import string
import MDAnalysis as mda
from sklearn.metrics import accuracy_score,f1_score
torch.set_grad_enabled(False)


## Helper Functions for ESM-1b

In [2]:
deletekeys = dict.fromkeys(string.ascii_lowercase)
deletekeys["."] = None
deletekeys["*"] = None
translation = str.maketrans(deletekeys)

def read_sequence(filename: str) -> Tuple[str, str]:
    """ Reads the first (reference) sequences from a fasta or MSA file."""
    record = next(SeqIO.parse(filename, "fasta"))
    return record.description, str(record.seq)

def remove_insertions(sequence: str) -> str:
    """ Removes any insertions into the sequence. Needed to load aligned sequences in an MSA. """
    return sequence.translate(translation)

def read_msa(filename: str, nseq: int) -> List[Tuple[str, str]]:
    """ Reads the first nseq sequences from an MSA file, automatically removes insertions."""
    return [(record.description, remove_insertions(str(record.seq)))
            for record in itertools.islice(SeqIO.parse(filename, "fasta"), nseq)]

def load_ESM_1b():
    """ Download the ESM_1b model and initalizes it."""
    esm1b, esm1b_alphabet = esm.pretrained.esm1b_t33_650M_UR50S()
    esm1b = esm1b.eval().cpu()
    esm1b_batch_converter = esm1b_alphabet.get_batch_converter()
    return esm1b,esm1b_batch_converter

## Data for Contact Map Analysis

#### The query for obtaining the list of PDB_IDs are as follows:
<a href="https://www.rcsb.org/search?request=%7B%22query%22%3A%7B%22type%22%3A%22group%22%2C%22logical_operator%22%3A%22and%22%2C%22nodes%22%3A%5B%7B%22type%22%3A%22group%22%2C%22logical_operator%22%3A%22and%22%2C%22nodes%22%3A%5B%7B%22type%22%3A%22group%22%2C%22nodes%22%3A%5B%7B%22type%22%3A%22terminal%22%2C%22service%22%3A%22full_text%22%2C%22parameters%22%3A%7B%22value%22%3A%22kinase%22%7D%7D%2C%7B%22type%22%3A%22terminal%22%2C%22service%22%3A%22full_text%22%2C%22parameters%22%3A%7B%22value%22%3A%22crystal%20structure%20%22%7D%7D%5D%2C%22logical_operator%22%3A%22and%22%7D%5D%2C%22label%22%3A%22full_text%22%7D%2C%7B%22type%22%3A%22group%22%2C%22nodes%22%3A%5B%7B%22type%22%3A%22group%22%2C%22nodes%22%3A%5B%7B%22type%22%3A%22terminal%22%2C%22service%22%3A%22text%22%2C%22parameters%22%3A%7B%22attribute%22%3A%22rcsb_entry_info.resolution_combined%22%2C%22operator%22%3A%22range%22%2C%22value%22%3A%5B1%2C1.5%5D%7D%7D%5D%2C%22logical_operator%22%3A%22and%22%7D%2C%7B%22type%22%3A%22group%22%2C%22nodes%22%3A%5B%7B%22type%22%3A%22terminal%22%2C%22service%22%3A%22text%22%2C%22parameters%22%3A%7B%22attribute%22%3A%22entity_poly.rcsb_entity_polymer_type%22%2C%22operator%22%3A%22exact_match%22%2C%22value%22%3A%22Protein%22%7D%7D%2C%7B%22type%22%3A%22terminal%22%2C%22service%22%3A%22text%22%2C%22parameters%22%3A%7B%22attribute%22%3A%22rcsb_entry_info.resolution_combined%22%2C%22operator%22%3A%22range%22%2C%22value%22%3A%5B1%2C1.5%5D%7D%7D%5D%2C%22logical_operator%22%3A%22and%22%7D%5D%2C%22logical_operator%22%3A%22and%22%2C%22label%22%3A%22text%22%7D%5D%7D%2C%22return_type%22%3A%22entry%22%2C%22request_info%22%3A%7B%22query_id%22%3A%2213e2ed91c000ae1a14db66b7028d506d%22%7D%2C%22request_options%22%3A%7B%22pager%22%3A%7B%22start%22%3A0%2C%22rows%22%3A25%7D%2C%22scoring_strategy%22%3A%22combined%22%2C%22sort%22%3A%5B%7B%22sort_by%22%3A%22score%22%2C%22direction%22%3A%22desc%22%7D%5D%7D%7D">Search Query</a>



In [3]:
query_res="3KHF,3LQ3,3FEG,6YA6,2WWE,2IIM,4L0N,3WAR,2X18,7A22,7A1Z,7A2H,7A1B,1WYX,6Z19,3M9J,4YPD,6ELM,7D0E,2YJC,6YZH,6GY5,5JJ2,7CCW,2HSH,6RCH,6RCG,1KGD,3TW2,3O46,3I4W,6TSE,3S4E,3G36,6EGE,2QT1,6HOG,2QFA,2C60,2XRW,5GJH,6G9Z,5O8I,1LKK,4N6X,3ZYQ,2C01,4RFZ,3L46,5EP6,1EAZ,6KC4,6KEC,6KEI,5K26,6TT2,4EIH,5NP5,6B9X,3FIA,4LMM,2R3R,2R3I,6HOL,6J5S,5ED3,6TEW,4FKL,4EK4,4EK3,4P5Q,6HMB,6HMC,6HMD,4LDJ,4OBE,3LNY,3WA4,1OBX,5P21,5XRD,4YDP,4FH0,7F0W,6ZBK,3EAZ,3EAC,6Q4Q,5AUL,7L9M,6SRH,7M10,6Z36,6B42,5WA8,5WA9,5LR1,5XRE,5NYI,3Q6L,4MPA,2R3Q,2WP3,4RRV,6I5H,7REK,5W55,2BRF,4GRZ,3CZT,4N19,6VJJ,6G36,6HOI,5J1S,2YJ2,2YJ9,2YJ8,2YJB,2IZX,5ZCK,4FKU,5J1T,2OVJ,6RQW,6RQQ,6J65,6DMJ,6HYO,7AJN,6SE4,6S25,5O97,6ZED,6ZEL,6ZF9,6I7Y,6I7X,5E0R,4JL7,4Q3H,5KJZ,3K05,6ZVO,4HYI,5Y90,2NZL,2PEZ,2HC1,2HC2,7CIO,5FTG,6Y1J,1XD3,6J5Z,3CJW,6TGK,6HMT,2GCO,6FT8,5LXI,6G39,6G3A,4XX9,3BBB,6G37,6G38,5W2F,2VIF,6FBB,5KLY,5KM4,5XCO,6AC0,6FNI,4C6K,4C6J,4C6I,4C6F,4C6E,4C6D,4C6C,6FNK,6FNL,6FNJ,4X3K,1NTE,5J80,5J64,7AEW,6FCP,5P9K,4HCV,4HCU,4HCT,3KZD,2RGG,6Y8E,2A2A,7A2N,6QUW,6QUV,2FE5,3O3U,7A2S,7A2T,7A2R,7A2Z,4XOF,3KNB,1M9Z,1WMA,6FNM,6ZJE,2YEX,7NDY,2UV4,3MHR,3K82,7AAH,4HZR,4E1H,7B8H,6NMW,6QUU,4QTC,5HHE,5IO9,1W70,6ZRN,4BB9,5YWR,1GP0,6XXX,1D4T,4ZX2,1W1G,6G6K,3IQJ,7NPG,7NPB,7NP2,7NND,3A7I,4UU5,6M64,3NSZ,4QLI,6T29,7LB1,3GU6,3GU4,2OSS,3EYI,3U5L,1W1H,3G4E,3UVW,5DC4,6M5Y,5VBM,4KS7,5NNF,6H5W,2A2R,2OUR,4Y5I,5AV4,4DHR,7L9G,1WMS,1H8D,5ZN0,5ZN1,5ZN2,5E8X,3IQV,3IQU,6MV4,5J2X,4D3P,5DQY,5EOP,3CO1,7LAY,1PWA,5MSA,5KM2,4PF4,3BHY,3OT3,2DKO,3AJ4,6N7A,6S17,6S1B,6S1J,6S1H,6S14,3PA3,5CVG,5CSV,5OGJ,3L8Z,3TGP,2AFY,6LCB,5NNG,5CU6,1I73,1TGR,7REE,6YIA,7LL4,5E8S,5DK8,5D7F,2GBJ,6V6U,6V6V,2WWZ,1OKI,1PA7,4KWP,3BUX,4WNJ,3OBQ,2RGB,2RGE,5MOH,3W80,3TT8,5QU7,5QU4,5QU3,6V6M,5S9B,5S9C,5S9A,5S9J,5S9H,5S9I,5S9F,5S9G,5S9D,5S9E,5S92,5S93,5S8R,5S90,5S91,5S8Y,5S8Z,5S98,5S8W,5S99,5S8X,5S8U,5S97,5S8V,5S94,5S8S,5S95,5S8T,3EY6,5ELQ,5OHH,7B13,4O71,6L1S,6LVQ,7C8S,6QJL,6QJK,5OUL,5OTQ,5OTY,5OTZ,6EHK,5H8X,1UZK,3P34,7S5A,4LM4,4LM3,2QUZ,1MFG,6BLN,6BKW,6PCL,6PCK,3SOV,5NK5,5NK6,5NK4,5NKA,5NKG,5NKH,5NKE,5NKF,5NKC,5NKD,4PQW,6FNG,6FNH,4RX5,6Q7C,6Q7B,6HEY,6Q7G,6Q7E,6Q7F,4HJP,3F66,5MXO,4WOH,1I76,5KUP,4XZF,6HEX,6HEV,6HEW,6HET,6HES,6Q3Y,4YLL,4YLK,4PS5,5QU1,6HRP,6HRT,6DI9,6DI5,5U9D,5E95,6DI0,6DI1,1KQ6,6X3P,3P1N,5L0S,4O7A,4O74,4O72,4O78,3TEM,6PKF,4CY7,6ICG,6TC2,4U7H,5NNE,5MME,1XOZ,1S1P,5OAZ,4Q26,5I9Y,6QHL,6UYZ,6V7S,6V7Q,5I9X,4UEH,5MYC,5MY9,5VBO,5I9V,7AV9,7BBO,4LYH,5WA5,6CJ2,6CD4,5I9W,1LF7,6GD3,6AC5,7L73,6J6M,6HFI,6HFE,3GJ0,4LUC,6HFU,6HFK,6HFL,6HFJ,6HFH,6HFS,6HFR,4X9J,5A1I,1TBF,6KUG,6E4F,6TJQ,4ZGG,3T0H,7RT1,7RT5,5P9M,5P9L,5P9I,5P9J,4J46,4J45,4J44,4J47,5S9K,5S8A,5S81,5S7N,5S7L,5S7M,5S7K,5S7Y,5S7W,5S85,5S7T,5S86,5S83,5S84,1WM3,1MH1,2VXT,5EMA,2B69,6HFP,6HFN,5F2E,7S6O,6OC0,6PHQ,6PHM,6PHN,3BER,3GHU,3GHT,3GHS,3GHR,3Q1I,3WHA,3B28,4QL3,4TQ9,4TQA,5YOV,6FQO,6FQU,5OWK,5EP7,5ENG,5MPN,5MMG,6O5W,6HFQ,6MJ7,5F62,5F63,5F60,5F61,4GA0,6W2G,4UFD,4M1Y,6MBU,4X2I,7C2Z,5LRL,6OLX,6N8X,4JV8,6FBW,5S7Z,5S7A,5S7H,5S7I,5S7F,5S7D,5S7E,5S7B,5S7C,5S7P,5S7O,5S7J,5S89,5S7X,5S87,5S7V,5S88,5S7R,5S7S,5S78,5S76,5S77,5S75,5V9U,6XXJ,6B0Y,5LVN,6FJT,5KM3,4J9E,4J9C,4BW1,6V0U,4J9F,7NFB,4CFL,4QC4,2X2A,6BOF,6P0J,1PIN,6P0K,6P0L,6HH2,3U2C,5KM9,4QBV,5KYC,6FAV,6FAW,6FAU,6PHO,6PHL,6PHI,4EO7,4ONM,5GOD,5GOB,5W7W,2OUS,6EDF,4QR5,4QR4,4QR3,4NUC,4NUE,4NUD,4UDW,4UE7,4UD9,5AHG,5AFY,5AF9,6FT3,6FT4,6FSY,6VUJ,4PCI,4PCE,6ZB3,6CZU,6PS9,6PRT,7O6K,7NSV,7O3A,7O3F,7O3P,7NR7,7NMH,7NLE,7NM3,7NK5,7NJB,7NK3,7BIQ,7BI3,7NYG,7NYE,7NZ6,7RXR,7RXS,4IOR,4IOO,4QZS,6AJV,6AJW,6VRD,6MBT,4J3I,5M39,4LYI,7MLS,7MLR,7MLQ,6YQO,6YQP,6YQN,5DW2,3T10,5D0C,4YH4,6E4A,5HM0,4Z1S,4Z1Q,6RWJ,6S6K,6U0D,6XUZ,6XVC,6XV3,6B0V,6G0R,4AW0,5J41,6ODG,6SZM,6G3R,6Y1E,5PYW,5PXB,5PXA,5PXL,5PY5,5PXT,5PXS,5PXN,5PWG,5PWF,5PX4,5PWS,5PX2,5PWR,5PX6,5PWU,5PAG,5PAV,5PAS,5PB2,5PAO,5PAX,5PAE,3T7L,6B1K,6B29,5V1Y,2GRR,5Z1S,5Z1T,6BCB,6CYH,5ENA,6HMU,2PPO,3BFO,6B26,2Y5F,2Y5H,2Y5G,5GWN,4EY1,4EY9,4EYD,2ERF,5QID,5QIE,5QIC,5QIH,5QIF,5QIG,2FGB,1ZQ5,5KYF,6YIM,6YIL,4NYW,4NYX,4NR7,5UG8,5UG9,7PBB,1Z70,3G9Y,6PHK,6LJW,6LJV,6LJT,4YUH,4YUG,4YUO,4YUL,4YUK,4YUJ,4YUI,6P7Z,6MBQ,7KTE,5WMJ,6GJ5,4XO6,4GQ4,4GPQ,4OG4,4OG6,6O53,4C9X,3EG3,5LMA,6DJC,5XHE,5KU3,5NWD,5NWG,7ROV,3I3S,5Y8C,6SQO,3RFE,4DLR,5KHM,6XDS,2FST,4WY4,5QU5,5HPR,5HGZ,6P0Z,6D31,2OMH,2ZHN,2FZD,4ONL,2X5Y,4JMG,5INB,4F3I,4YLQ,6R2W,2CLD,2CLC,2CL7,2CE2,2I5F,5GH9,2W73,1R8S,5V37,6TX8,4U1M,3ZM1,2HE4,5QU2,1P5F,2FR3,4J3Y,4UP0,4GRN,6G0S,2FZ8,7KT5,1CTQ,3BUW,7K5L,6G0P,4WJQ,5OCG,5M9O,2CL6,2EVW,6G0Q,3RS0,5C2K,7NEL,6G0O,5XCT,5XCQ,7KHL,5JFD,6T56,6T55,6T53,6YQV,6YHG,5A2M,6TKL,6TKG,6Y02,6VAJ,2ZQT,3Q27,6IPY,5F6Y,5F6V,5F6E,6HXU,4DRI,5HG8,4GR8,3PP2,6PAF,6G2N,4KNN,2ITK,2E3N,2E3P,7AVC,7ER8,2C9S,2C9U,2C9V,4HY4,1DG6,4AG1,3NCL,7KG8,4QUJ,5CQ2,3SJM,6QW3,6NV2,6QHM,5MM6,5MJT,5I88,5KTU,4FYO,5MPZ,5MQG,6I44,5B4O,5TIU,4PZ3,6RL3,5R8B,5R8A,5R8K,5R8I,5R8F,5R8G,5R8D,5R8E,5R8Q,5R8N,5R8O,5R8M,5R88,5R87,5R85,5UQA,1MFM,5COY,3UGC,3TOW,6O24,4JJE,3VXE,1OGW,1Z89,6TNQ,7NHU,2H3L,5NI9,5NIG,5NR4,4IUM,4N7F,5YQO,3SI4,6T4A,6T3Q,5LCE,3RMO,3RM2,3RM0,3QX5,3SV2,3T5F,5NL4,2UUJ,2UUF,2UUK,2CN0,3SXE,6Y3V,3H61,3H63,5UGG,5UGD,3H62,1I27,4IGH,6I42,5HKJ,3AP9,4LNP,4LN2,6BCD,5L0T,4NIB,5HRQ,1BKR,5TXY,5KFS,5KFN,7M7N,7M7M,7M84,7M7T,4HF3,6P0I,7RNU,6NSV,7NX1,5S8I,5S8J,5S8G,5S8H,5S8E,5S8F,5S8C,5S8D,5S8P,5S8M,5S8N,5S8K,5S8L,5RKU,5RKV,5RKS,5RKT,5RKQ,5RKR,5RKO,5RKP,5RKY,5RKW,5RKX,5RK5,5RJT,5RK6,5RJU,5RK3,5RJR,5RK4,5RJS,5RK1,5RJP,5RK2,5RJQ,5RJN,5RK0,5RJO,5RJZ,5RK9,5RJX,5RJY,5RK7,5RJV,5RK8,5RKE,5RKF,5RKC,5RKD,5RKA,5RKB,5RKM,5RKN,5RKK,5RKL,5RKI,5RKJ,5RKG,5RKH,5RJL,5RJM,5RJJ,5RJK,5ENE,5ENF,4DJC,4DAT,4INC,3LW1,1UNR,1P1V,2VR6,1OZU,3SXB,3SX8,3SX5,6LJE,4EEW,4WZX,2IKG,2IKI,7JGW,4GV1,6KX2,3WLU,4ICV,4OQV,6HYN,3CBP,4WK9,4WKH,4WKF,6XLV,4WJO,5UDP,5E50,1X96,1X98,3ONB,3ONC,2QJZ,4UZ1,4WW6,6XXC,6XY5,6Y18,6UYV,5KFZ,7MU3,5SYA,5SY9,5SY6,4P2G,3CZA,7AA7,6V1B,7RNS,6MX3,3HJ0,5MHC,4HI9,4HI8,4FRA,7B37,7ARG,7B3F,5W46,4P36,3CZ9,3CY6,2F3Y,7NYO,7NYK,5UUI,7QHG,4QSB,6Y3W,6Y1D,4PZH,4Q08,5N5W,3MC5,3M64,3M0I,3LEN,7O07,4DO4,4MZI,6FYR,3F1P,2IZR,7BRQ,5JNT,6L0X,6F4Q,7LIP,6D30,6D2Z,5JIX,6RJL,6RHC,7BF2,5NXI,6EIM,5JGI,5C7G,4U3Y,5CMM,6R8R,6GRO,2UYZ,7ABT,1QTN,1ZD8,4YPC,4YBM,5KH9,5B8D,4JJ7,3EWY,3EWX,4GAI,7AZ1,7AZ2,7AXN,7BG3,3MRK,5KTW,5KTX,6RA7,4NFN,6FEW,3DN5,5J0D,1T41,5MFA,5L0N,6TUZ,4OUF,2PIE,4QX5,2W4J,5I83,5I89,5I86,2WZB,3ZZY,4GWG,6MYE,5JAX,5J48,2QOC,3FED,2QOI,2QOK,2QOL,2QOO,2QOD,2QOF,5OQ5,6HB9,6FEX,3U9W,7L8Q,6B6U,3DJH,6N6K,5G4M,5G4N,7BF1,6FVF,3F1N,2VR8,7AT9,6ZEI,5FAF,6QHJ,6G8L,6G8J,6G8K,6G6X,6GUK,1MKK,6TLS,5QIT,5QIR,5QIQ,5QIO,5QIX,5QIV,3HSV,6AY5,4R1V,7SUG,7SUF,4P2K,3ZCL,6XZZ,6XYX,7PD9,5THN,6Q4F,6Q3F,6Q3C,6OMU,6Q4H,6Q4I,6Q4D,6Q4E,6Q4B,6Q4J,6Q4K,6Q3B,6Q48,6Q49,4HBQ,3MI2,6Q4A,5A7G,5YP8,6FCH,2VZC,6ESM,5LCP,7B9T,7B9R,7BA8,7BA7,7BA9,7BAB,7BAA,7BA3,7BA6,7BA5,7NIZ,7NFW,5M6V,6O8I,6G5P,7KSW,5M71,5M6Y,5M0L,5LCQ,5LBZ,5LBY,7L67,7L64,7N5O,7KXM,7KXQ,7KXN,1H10,6OSH,6OSN,6OSV,4GCJ,6E28,1GK7,1Z0J,5MHQ,5TZM,5EHW,5EHV,6R8P,6TJT,6ZTB,7KT0,7KSZ,5KO0,6VXQ,4CT2,6NFH,5ACK,5YVN,1GZ8,7B2V,7B2Y,5LVL,5YP7,3SX7,3SX3,5LVM,6SI3,6SI1,6SHZ,7OFV,4XCT,5W3R,3CBZ,6O4J,5WAA,2V0A,6ZK0,3I0G,5ZMZ,6YQM,5BXC,5C49,5LVO,3FEC,7PB5,7P9R,5LO5,5MRD,2YI7,3NHE,1LZI,1LZJ,7RXV,4XJJ,6TN5,6TN4,6W2I,3MYQ,3D06,3D08,7LIN,6YHA,5G4O,6V4F,5LL4,6N6I,1RG8,4RI5,6YQ2,6YPY,6YOW,7O6M,7O6J,7NV4,7O6O,7O57,7O59,7NQP,7O34,7NLA,7NM1,7NJ9,7BKH,7BJF,7BJL,7BJW,7BIW,7NZK,7NZG,7NZV,7NYF,7NXW,7NXY,7NY4,7NXS,7NXT,7NWS,7NVI,4N12,7KT2,4OQ4,5JO3,7KTG,7KTN,7KT9,3EZG,1SOA,4BCY,6YOJ,7LH7,3SP7,4X2F,6N3X,6N3V,3OIU,3OIW,4EPV,6GPO,7RPZ,1UYL,5BN8,6PQA,7A1X,6MS9,3VHA,2CIA,4BTE,4PQD,6E5Z,4S0Z,4MTC,3VHC,2YKJ,2YKE,2YK9,3O0I,2OR3,3F71,6GQS,6GR5,2GZV,6AX4,4N1M,4N1S,6HXM,6HXL,4Z93,3WUZ,4LR6,5KMC,4Q27,4Q2F,4Q1R,4Q1P,5MWX,6G0D,7NZD,7NZC,5K7N,5DRS,3FF3,6QL2,6QL3,6O5I,6MQT,1L3K,4U1P,5M33,6T5B,6T5V,6TAN,4QTA,6ZCS,5J0F,4PZZ,4Q03,4Q01,4X9V,4X9R,6G0G,6BQA,4ZZN,3K0N,3K0M,5TA2,5T9Z,5V5C,6GGF,5O1I,6GGC,6GGD,6GGB,5O1G,5O1D,5O1E,5O1B,5O1A,5O1H,5AOK,5LUD,5NOY,5NOW,5NOX,5NOU,5NOS,5NOT,6TKK,6SLG,6X3Y,2X25,5WC7,5F66,4PWQ,3GP2,5MYX,5WP1,6GJL,6GJM,6GJJ,6GJY,6GS6,6AR0,4QTB,4YJR,4YJQ,5I2F,7B4N,7B49,7B4H,5UF0,6GBI,5AD3,4O6W,6YQZ,4HXS,4HXN,5CFW,6AJZ,5EI4,6SWN,4UIZ,4UIY,4CL9,6Z7M,6DL2,5A5S,1T7H,4UYD,6UWX,6UVJ,4CFQ,6TS3,7JKY,7JKX,7JKW,5LJ2,6FNX,7A9U,5XI4,6JJ5,6JJ6,7M16,7P6W,7P6V,4WHW,5S96,4Q7Z,5QS8,5QS9,5QSJ,3U2U,7LCU,3P6C,1X97,4XZH,4QX4,1PWL,4OEL,5IPC,5IPE,4GGC,3K8Y,4AOH,4GRR,4GRP,6YF2,6YF3,6YF1,5M9Q,5M9R,5MG7,5MCT,5WDQ,5WDP,6RWU,5H5Q,5H5R,4TNS,2RJ5,4A3P,7AC9,5EDC,5EDB,6YVK,6YVN,6YVL,6YVM,5I80,5HZ6,5HZ8,5N20,3RZY,3P6H,3P6G,3P6F,3P6E,3P6D,7KTF,5Z02,5ZXF,6FVH,6FVE,6TPZ,6TPX,4CFR,3I6C,3IJJ,5WMA,5WMG,5WMD,3TC5,5HGJ,6GX2,5NJ9,5NJA,6T52,6T3M,3U98,6I51,6YSJ,6YN3,6YMP,6YHJ,6YB6,6Y9H,5IBP,6GMC,6GMB,3EXX,3W7Z,6VET,4X5W,6S34,6VER,4XC4,5USV,5UU2,5URT,5USP,5UOZ,3U8R,4AJX,4R0U,6GV0,5I29,2BVR,2BVS,2ZFF,6OD3,4ZNN,4RIL,5C4M,2JKH,4QKZ,2J30,2J32,3KL6,5SVY,4FKA,6GBW,6ROT,5JZY,3P17,5R7W,6Y8I,4HK2,5NHU,6YSX,4IBY,5A7B,4AGQ,4AGO,3U8O,1HE4,5BT3,6NE2,5WL1,4GQR,4Z47,1C5L,2CF8,6SBO,6VIG,5C0F,5NVG,1J96,5L0V,2I7D,5TXD,3PZZ,3Q2X,5TOG,7CAP,4JTR,4JQA,6D3Y,5VOS,7KJM,1KJL,6UYR,6UYP,6OIZ,6NB9,5TOF,6T5G,6T27,6IBE,6X8H,4PUW,6TUF,6TUC,2AXI,6YBM,4Q2Q,6K39,5ONQ,5ONR,5ONP,5HOB,4D6A,6Q0M,6Q0N,6F4P,2RK6,5HOC,6T7Q,2AGT,3T42,3LZ3,3LQL,3LD5,3LBO,4NIN,6F8O,6F84,6F82,4OWZ,6BXX,2RK4,2RK3,2R8U,5EH7,5EH8,5EH5,6AFC,6AFJ,6AFD,6AF9,6AF7,2WD2,4CQ0,6GL8,5RJI,3MRG,6ZEG,5UL6,3D25,2WNV,4NIO,3UVC,5YBY,4ZSD,5CBT,6ALC,6F78,5I8G,5V8Q,6AY3,6AXQ,5W0E,6RT6,6RT4,6EW6,5ZGD,4GYX,6UGO,6I3E,6X8O,6E78,6E72,6T2K,6T2H,6SJA,7B9N,7B9I,7B84,7B7Y,7B86,7B8C,7B8A,7B8L,7B8M,7B45,7B3G,7BDA,7BDF,7BDC,7BDB,7BDD,7BD8,6ZVL,6YXI,6PJV,3SGP,6YSK,6TR7,6TR6,4OHA,5C5A,3ZME,5TZL,4A7V,4A7T,4A7S,4A7Q,4A7G,2WYT,6Z3V,6Z4G,6Z4O,6Z4K,5O3Y,6GGE,5O1F,5O1C,5AOJ,5AB9,5LAP,6MS7,2XJK,4WT2,4OGN,7KKN,2IFQ,3GZQ,5O5T,5OF0,4RMI,4RMH,4GQQ,2RJ4,4AVS,3D32,7AES,5EXN,2PZN,4QR6,4Q7B,4PUU,4RPQ,4YU1,4PRR,4PR4,4NKC,2PQ8,6YIP,4M5S,1RYO,4YS1,6D40,2PDG,2PDH,2PDL,1Z3N,3LQG,5KL3,4PK3,4PK2,4Q6S,2XG3,4UE1,1A7S,6HN3,7NUS,6Q9L,6Q9O"

In [4]:
pdb_list = query_res.split(",")

### Download corresponding PDB Files and store it in a directory

In [ ]:
pdblist = biopy()
PDB_IDs = pdb_list
counter = 1
for PDB_ID in PDB_IDs:
    print(f'At entry {counter} out of {len(PDB_IDs)} entries.')
    pdblist.retrieve_pdb_file(pdb_code = PDB_ID, file_format = 'pdb', pdir = './data1/test1/')
    counter += 1

### Download corresponding Files and store it in a directory

<a href="https://www.rcsb.org/downloads/fasta">Use this link to download all FASTA files by providing the list of PDB IDs</a>



## Comparative Analysis of Contact maps- ESM_1b  and PDB Structure

In [5]:
def getContactMap_fromPDB(PDB_file):
    """ Takes a PDB file and creates a PCN from the alpha carbons and computes distance matrix 
    followed by generating a contact map with 8.0 Å threshold value."""
    protein_contact_network = PCN.PCN(PDB_file)
    C_alphas = protein_contact_network.get_C_alphas()
    chain_length = protein_contact_network.get_chain_length(C_alphas)
    if protein_contact_network.get_link_lengths(C_alphas) is not None:
        distance_matrix=protein_contact_network.get_link_lengths(C_alphas)
        contact_map = distance_matrix >= 8
        return contact_map
    else:
        return None

In [6]:
def getContactMap_fromESM(esm1b,esm1b_batch_converter,fasta_file):
    """ Loads the ESM_1b model, computes contact map from the inputted FASTA file. """
    esm1b_data = [read_sequence(fasta_file),]
    esm1b_batch_labels, esm1b_batch_strs, esm1b_batch_tokens = esm1b_batch_converter(esm1b_data)
    esm1b_batch_tokens = esm1b_batch_tokens.cpu()
    esm1b_contacts = esm1b.predict_contacts(esm1b_batch_tokens).cpu()
    seqlen = len(esm1b_batch_strs[0])
    #contact_map=esm1b_contacts[0][:seqlen, :seqlen]>=0.1
    contact_map=esm1b_contacts[0][:seqlen, :seqlen]>=0.1
    contact_map=contact_map.numpy()
    return contact_map

In [10]:
def compareContactMaps(contact_pdb,contact_esm,pdb_name,results_path):
    """ Compare contact maps generated by ESM_1b vs PDB and compute accuracy."""
    print(contact_pdb.shape[0],contact_esm.shape[0])
    if contact_pdb.shape[0]==contact_esm.shape[0]:
        contact_pdb_upperTraingle=contact_pdb[np.triu_indices(contact_pdb.shape[0],k=1)]
        contact_pdb_upperTraingle=np.multiply(contact_pdb_upperTraingle,1)
        contact_esm_upperTraingle=contact_esm[np.triu_indices(contact_esm.shape[0],k=1)]
        contact_esm_upperTraingle=np.multiply(contact_esm_upperTraingle,1)
        print(len(contact_pdb_upperTraingle),len(contact_esm_upperTraingle))
        acc,f1=accuracy_score(contact_pdb_upperTraingle,contact_esm_upperTraingle),f1_score(contact_pdb_upperTraingle,contact_esm_upperTraingle)
        title_text='Analysis of Contact Maps- %s'%(pdb_name)
        fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))
        fig.suptitle(title_text, fontsize=14,fontweight='bold')
        ax[0].title.set_text('PDB -> Contact Map')
        ax[0].imshow(contact_pdb,cmap="Blues")
        ax[1].title.set_text('ESM-1b -> Contact Map')
        ax[1].imshow(contact_esm,cmap="Blues")
        resultstr = 'Metrics\nAccuracy:%.2f'%(acc)
        ax[1].text(contact_esm.shape[0]+15, contact_esm.shape[0]/2+15, resultstr, fontsize=12)
        fig.tight_layout()
        final_result_path=results_path+pdb_name+".pdf"
        fig.savefig(final_result_path, bbox_inches='tight')
    else:
        print('FASTA and PDB are differnet for' +str(pdb_name))

In [14]:
def main():
    PDB_IDs = pdb_list
    #Load ESM_1b Model
    esm1b,esm1b_batch_converter=load_ESM_1b()
    #Set the path to results
    results_path="/Users/rohan/Desktop/DTI_setup/contact_results/" 
    for i,j in enumerate(PDB_IDs):
        print(PDB_IDs[i])
        PDB_IDs_lower = str(PDB_IDs[i]).lower()
        #Set path to PDB files
        PDB_file = f'//Users/rohan/Desktop/DTI_setup/data_main/pdb_files/pdb{PDB_IDs_lower}.ent' 
        #Set path to FASTA files
        fasta_file=f'//Users/rohan/Desktop/DTI_setup/data_main/fasta_files/{PDB_IDs[i]}_1.fasta'
        if os.path.isfile(PDB_file) and os.path.isfile(fasta_file):
            contact_pdb=getContactMap_fromPDB(PDB_file)
            contact_esm=getContactMap_fromESM(esm1b,esm1b_batch_converter,fasta_file)
            if contact_pdb is not None:
                compareContactMaps(contact_pdb,contact_esm,PDB_IDs[i],results_path)
            else: 
                continue

In [ ]:
main()

3KHF
106 99
FASTA and PDB are differnet for3KHF
3LQ3


/Users/rohan/opt/anaconda3/envs/mldd/lib/python3.8/site-packages/MDAnalysis/topology/PDBParser.py:334: UserWarning: Unknown element X found for some atoms. These have been given an empty element record. If needed they can be guessed using MDAnalysis.topology.guessers.
  warnings.warn(wmsg)
/Users/rohan/opt/anaconda3/envs/mldd/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: X
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))


In [1]:
from tdc.multi_pred import DTI
data = DTI(name = 'KIBA')
split = data.get_split()

ModuleNotFoundError: No module named 'tdc'